Group 13 | Aditya Sharma, Adarsh Balan, Thaneshwar Prasad Sahu, Muhammad Ashraf Hussain, Prathyusha Thatipelli
Part B | Question 3

In [1]:
# Connecting to Google Collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importing Libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, classification_report

In [3]:
# Checking if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# Loading the dataset
train_dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/AAI Group Assignment/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/AAI Group Assignment/test', transform=transform)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
# Loading pre-trained VGG19 model and modify it
model = models.vgg19(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = torch.nn.Linear(num_ftrs, 15)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:04<00:00, 119MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [8]:
# Defining loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [17]:
# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/25, Loss: 0.00029038841603323817
Epoch 2/25, Loss: 0.003390142461284995
Epoch 3/25, Loss: 0.001954454928636551
Epoch 4/25, Loss: 0.0009592715068720281
Epoch 5/25, Loss: 0.0007679020636714995
Epoch 6/25, Loss: 2.6104991775355302e-05
Epoch 7/25, Loss: 0.0002920116239693016
Epoch 8/25, Loss: 7.186943548731506e-06
Epoch 9/25, Loss: 0.00355253997258842
Epoch 10/25, Loss: 0.00022705852461513132
Epoch 11/25, Loss: 4.281125438865274e-05
Epoch 12/25, Loss: 6.314789061434567e-05
Epoch 13/25, Loss: 0.0011150952195748687
Epoch 14/25, Loss: 2.4520792067050934e-05
Epoch 15/25, Loss: 1.689718737907242e-05
Epoch 16/25, Loss: 0.0006182850920595229
Epoch 17/25, Loss: 0.0030808441806584597
Epoch 18/25, Loss: 0.00032424964592792094
Epoch 19/25, Loss: 0.007072964683175087
Epoch 20/25, Loss: 0.00010252150968881324
Epoch 21/25, Loss: 6.611769640585408e-05
Epoch 22/25, Loss: 5.265396976028569e-05
Epoch 23/25, Loss: 3.058983929804526e-05
Epoch 24/25, Loss: 6.584721995750442e-05
Epoch 25/25, Loss: 7.923

In [18]:
# Evaluation
model.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

In [19]:
precision = precision_score(all_labels, all_preds, average=None)
recall = recall_score(all_labels, all_preds, average=None)

In [20]:
# Get class names
class_names = train_dataset.classes

In [21]:
# Printing precision and recall for each class
print("Per-class Precision and Recall:")
for i, class_name in enumerate(class_names):
    print(f"{class_name}: Precision - {precision[i]:.4f}, Recall - {recall[i]:.4f}")

Per-class Precision and Recall:
accordion: Precision - 1.0000, Recall - 1.0000
bass: Precision - 1.0000, Recall - 1.0000
camera: Precision - 1.0000, Recall - 1.0000
crocodile: Precision - 0.7778, Recall - 0.7000
crocodile_head: Precision - 0.7500, Recall - 0.8182
cup: Precision - 1.0000, Recall - 0.9412
dollar_bill: Precision - 0.9231, Recall - 1.0000
emu: Precision - 1.0000, Recall - 1.0000
gramophone: Precision - 1.0000, Recall - 1.0000
hedgehog: Precision - 1.0000, Recall - 0.9286
nautilus: Precision - 0.9375, Recall - 1.0000
pizza: Precision - 1.0000, Recall - 1.0000
pyramid: Precision - 1.0000, Recall - 1.0000
sea_horse: Precision - 1.0000, Recall - 1.0000
windsor_chair: Precision - 1.0000, Recall - 1.0000
